## Ejercicio 2: Diseño y construcción de modelo data warehouse. (4pts)
**Descripción**: Se plantea diseñar un modelo data warehouse sencillo a partir del resultado del ejercicio 1, construido directamente con notebooks jupyter, para ello el modelo debe satisfacer las siguientes necesidades:
- Productos más vendidos
- Clientes con el mayor número de pedidos
- Corresponsal con el mayor número de pedidos
- Total de pagos diario y mensual por productos
- Total de pagos diario y mensual por clientes
- Total de pagos diario y mensual por corresponsal

**Requisitos:**
- Diseñar un modelo de data warehouse de tipo estrella, incluir tablas de hechos y dimensiones que satisfagan los reportes de BI
- Construcción de diagrama data warehouse.

**Entregables:**
Incluir dentro del proyecto:
- Diagrama de data warehouse (1p)
- Notebook Jupyter que realice lo siguiente:
    - Limpieza de datos: (2p)
        - Nombres en mayúsculas, sin tildes, sin ñ, sin guiones bajos y medios.
        - Valores numéricos con solo dos decimales redondeados al inmediato superior.
    - Integración de los datos: (1p)
        - Scripts con la creación de las tablas de dimensiones
        - Scripts con la creación de las tablas de hechos

In [85]:
!pip install sqlalchemy psycopg2-binary pandas

In [87]:
#Instalacion de libreriras
from pyspark.sql import SparkSession
from sqlalchemy import create_engine, text
import pandas as pd

In [89]:
# Crear sesión de Spark
spark = SparkSession.builder \
    .appName("DataWarehouseDW") \
    .config("spark.jars", "postgresql-42.7.5.jar") \
    .config("spark.driver.extraClassPath", "postgresql-42.7.5.jar") \
    .getOrCreate()
# Verifica que la sesión de Spark se haya creado
print(spark)

In [63]:
#Conexión a la base de datos de postgres
usuario = 'postgres'       
contrasena = 'postgres'  
host = 'localhost'         
puerto = '5432'           
bd = 'mp3_db'   

conexion = f'postgresql+psycopg2://{usuario}:{contrasena}@{host}:{puerto}/{bd}'

engine = create_engine(conexion)

In [64]:
# carga de datos de clientes
df_clientes = pd.read_sql('SELECT * FROM clientes', engine)
df_clientes.head()

,id_cliente,nombre,email,telefono,direccion,fecha_registro
0,1,Rosalinda del Vigil,maxi91@gmail.com,+34 696 109 211,Urbanización Ileana Jara 88 Apt. 84 Guadalaja...,2025-04-29 21:07:17.855258
1,2,Xavier Zaragoza Alarcón,cuestafortunato@gmail.com,+34737892325,"Pasadizo Ángel Acevedo 58 Apt. 55 Baleares, 3...",2025-04-29 21:07:17.855258
2,3,Micaela Borrego Borrell,ana-belenarrieta@rubio-querol.es,+34 717 22 88 8,"Pasadizo Edmundo Reguera 83 Apt. 68 Alicante,...",2025-04-29 21:07:17.855258
3,4,Claudio Gutierrez Salinas,aaron86@cruz.com,+34 962 07 59 1,"Glorieta de Arturo Amo 17 León, 79725",2025-04-29 21:07:17.855258
4,5,Vasco Parra,violeta58@yahoo.com,+34709764488,"Vial Roldán Solís 53 Navarra, 65772",2025-04-29 21:07:17.855258


## *Limpieza de datos: (2p)*

- Nombres en mayúsculas, sin tildes, sin ñ, sin guiones bajos y medios.
- Valores numéricos con solo dos decimales redondeados al inmediato superior.

In [78]:
#Importación de librerias

import unicodedata
import numpy as np

In [79]:
# Limpieza de texto
def limpiar_texto(texto):
    if pd.isnull(texto): return texto
    texto = texto.upper().replace("_", " ").replace("-", " ")
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = texto.replace('Ñ', 'N')
    return texto

In [81]:
# Limpieza de teléfono
def limpiar_telefono(telefono):
    if pd.isnull(telefono):
        return telefono
    telefono = telefono.replace(" ", "")
    return telefono

In [82]:
# Limpieza a Clientes
df_clientes = pd.read_sql('SELECT * FROM Clientes', engine)
df_clientes['nombre'] = df_clientes['nombre'].apply(limpiar_texto)
df_clientes['email'] = df_clientes['email'].apply(limpiar_texto)
df_clientes['telefono'] = df_clientes['telefono'].apply(limpiar_telefono)

df_clientes.head()


,id_cliente,nombre,email,telefono,direccion,fecha_registro
0,1,ROSALINDA DEL VIGIL,MAXI91@GMAIL.COM,+34696109211,Urbanización Ileana Jara 88 Apt. 84 Guadalaja...,2025-04-29 21:07:17.855258
1,2,XAVIER ZARAGOZA ALARCON,CUESTAFORTUNATO@GMAIL.COM,+34737892325,"Pasadizo Ángel Acevedo 58 Apt. 55 Baleares, 3...",2025-04-29 21:07:17.855258
2,3,MICAELA BORREGO BORRELL,ANA BELENARRIETA@RUBIO QUEROL.ES,+3471722888,"Pasadizo Edmundo Reguera 83 Apt. 68 Alicante,...",2025-04-29 21:07:17.855258
3,4,CLAUDIO GUTIERREZ SALINAS,AARON86@CRUZ.COM,+3496207591,"Glorieta de Arturo Amo 17 León, 79725",2025-04-29 21:07:17.855258
4,5,VASCO PARRA,VIOLETA58@YAHOO.COM,+34709764488,"Vial Roldán Solís 53 Navarra, 65772",2025-04-29 21:07:17.855258


In [68]:
# Limpieza a Productos
df_productos = pd.read_sql('SELECT * FROM Productos', engine)
df_productos['nombre'] = df_productos['nombre'].apply(limpiar_texto)
df_productos['descripcion'] = df_productos['descripcion'].apply(limpiar_texto)
df_productos.head()

,id_producto,nombre,descripcion,precio,stock
0,1,LAPTOP LENOVO THINKPAD X1 N CARBON,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: LAPTOP L...,442.47849,91
1,2,SMARTPHONE SAMSUNG GALAXY N S24,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: SMARTPHO...,272.04082,96
2,3,TABLET APPLE IPAD N PRO,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: TABLET A...,182.13936,20
3,4,MONITOR LG ULTRAWIDE N 34,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: MONITOR ...,232.92347,81
4,5,AURICULARES SONY WH 1000XM5 N WIRELESS,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: AURICULA...,496.58918,79


In [69]:
# Redondear valores numéricos de precio a dos decimales
df_productos['precio'] = np.ceil(df_productos['precio'] * 100) / 100
df_productos.head()

,id_producto,nombre,descripcion,precio,stock
0,1,LAPTOP LENOVO THINKPAD X1 N CARBON,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: LAPTOP L...,442.48,91
1,2,SMARTPHONE SAMSUNG GALAXY N S24,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: SMARTPHO...,272.05,96
2,3,TABLET APPLE IPAD N PRO,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: TABLET A...,182.14,20
3,4,MONITOR LG ULTRAWIDE N 34,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: MONITOR ...,232.93,81
4,5,AURICULARES SONY WH 1000XM5 N WIRELESS,PRODUCTO TECNOLOGICO DE ALTA CALIDAD: AURICULA...,496.59,79


In [70]:
# Se realzia tambien para Detalles_Orden y Ordenes
df_detalles = pd.read_sql('SELECT * FROM Detalles_Orden', engine)
df_ordenes = pd.read_sql('SELECT * FROM Ordenes', engine)


In [71]:
for col in ['precio_unitario', 'subtotal']:
    df_detalles[col] = np.ceil(df_detalles[col] * 100) / 100

df_ordenes['total'] = np.ceil(df_ordenes['total'] * 100) / 100

df_detalles.head()

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,subtotal
0,1,1,25,2,259.72,519.43
1,2,1,14,1,383.64,383.64
2,3,2,26,2,380.97,761.94
3,4,2,23,4,338.53,1354.09
4,5,3,28,4,184.53,738.11


In [72]:
df_ordenes.head()

,id_orden,id_cliente,id_corresponsal,fecha_orden,total
0,1,1,6,2025-04-25 01:12:43,903.07
1,2,1,1,2025-02-11 15:05:19,2116.03
2,3,1,4,2025-01-13 15:15:20,852.63
3,4,2,3,2025-03-24 04:18:00,1557.02
4,5,2,2,2025-02-09 22:28:48,1385.16


## **Creación de la base de datos para el DatawereHouse**

In [92]:
#Credenciales para el datawerehouse        
bd_dw = 'postgres'   
conexion_dw = f'postgresql+psycopg2://{usuario}:{contrasena}@{host}:{puerto}/{bd_dw}'
engine_dw=create_engine(conexion_dw)

In [93]:
# 4. Crear tablas en PostgreSQL
script_sql = """

CREATE TABLE IF NOT EXISTS TH_Ordenes (
    ID_Hecho SERIAL PRIMARY KEY,
    Fecha_Key DATE NOT NULL,
    Cliente_Key INT NOT NULL,
    Producto_Key INT NOT NULL,
    Corresponsal_Key INT NOT NULL,
    Cantidad INT NOT NULL,
    Precio_Unitario NUMERIC(10, 5) NOT NULL,
    Subtotal NUMERIC(10, 5) NOT NULL
);

CREATE TABLE IF NOT EXISTS Dim_Clientes (
    Cliente_Key INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Email VARCHAR(100),
    Telefono VARCHAR(15),
    Direccion TEXT
);
CREATE TABLE IF NOT EXISTS Dim_Productos (
    Producto_Key INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Descripcion TEXT,
    Precio NUMERIC(10, 5)
);
CREATE TABLE IF NOT EXISTS Dim_Corresponsales (
    Corresponsal_Key INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Direccion TEXT
);

CREATE TABLE IF NOT EXISTS Dim_Fecha (
    Fecha_Key DATE PRIMARY KEY,
    Anio INT,
    Mes INT,
    Dia INT
);

"""
# 5. Ejecutar el script SQL en PostgreSQL
with engine_dw.connect() as conn:
    conn.execute(text(script_sql))
    conn.commit()

print("✔ Tablas creadas exitosamente en el Data Warehouse.")

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near ")"
LINE 40: );
         ^

[SQL: 
CREATE TABLE IF NOT EXISTS TH_Ordenes (
    ID_Hecho SERIAL PRIMARY KEY,
    Fecha_Key DATE NOT NULL,
    Cliente_Key INT NOT NULL,
    Producto_Key INT NOT NULL,
    Corresponsal_Key INT NOT NULL,
    Cantidad INT NOT NULL,
    Precio_Unitario NUMERIC(10, 5) NOT NULL,
    Subtotal NUMERIC(10, 5) NOT NULL
);

CREATE TABLE IF NOT EXISTS Dim_Clientes (
    Cliente_Key INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Email VARCHAR(100),
    Telefono VARCHAR(15),
    Direccion TEXT
);
CREATE TABLE IF NOT EXISTS Dim_Productos (
    Producto_Key INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Descripcion TEXT,
    Precio NUMERIC(10, 5)
);
CREATE TABLE IF NOT EXISTS Dim_Corresponsales (
    Corresponsal_Key INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Direccion TEXT
);

CREATE TABLE IF NOT EXISTS Dim_Fecha (
    Fecha_Key DATE PRIMARY KEY,
    Anio INT,
    Mes INT,
    Dia INT,
    Nombre_M